In [19]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

ufo_url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv"
df = pd.read_csv(ufo_url, low_memory=False)
df.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


In [20]:
df_clean = df.dropna(subset=['Square Footage'])
df_clean.columns

Index(['Agency Name', 'Location Name', 'Address', 'City', 'Zip code', 'County',
       'Congress Dist', 'Congressional Full Name', 'Rep Dist', 'Rep Full Name',
       'Senate Dist', 'Senator Full Name', 'Bldg Status', 'Year Acquired',
       'Year Constructed', 'Square Footage', 'Total Floors',
       'Floors Above Grade', 'Floors Below Grade', 'Usage Description',
       'Usage Description 2', 'Usage Description 3'],
      dtype='object')

In [14]:
df_agency = (
    df_clean.groupby('Agency Name')['Square Footage']
             .sum()
             .sort_values(ascending=False)
             .head(10)
             .reset_index()
)
df_agency

,Agency Name,Square Footage
0,University of Illinois,25018006
1,Department of Corrections,15120750
2,Southern Illinois University,8709473
3,Department of Human Services,8466774
4,Department of Transportation,5659737
5,Department of Military Affairs,4579470
6,Department of Central Management Services,4260911
7,Department of Natural Resources,3937319
8,Northern Illinois University,3751095
9,Illinois State University,2960272


In [21]:
chart1 = (
    alt.Chart(df_agency)
    .mark_bar()
    .encode(
        x=alt.X('Agency Name', sort='-y', title='Agency'),
        y=alt.Y('Square Footage', title='Total Square Footage'),
        color=alt.Color('Agency Name', legend=None),
        tooltip=['Agency Name', 'Square Footage']
    )
    .properties(
        width=700,
        height=400,
        title='Top 10 Illinois Agencies by Total Building Area'
    )
)
chart1

alt.Chart(...)

In [22]:
df2 = df.dropna(subset=["Year Constructed"])
df2 = df2[df2["Year Constructed"] >= 1850]
df2["Usage Description"] = df2["Usage Description"].fillna("Unknown").str.strip()
df_year_usage = (
    df2.groupby(["Year Constructed", "Usage Description"])
       .size()
       .reset_index(name="Count")
)
df_year_usage.head()

,Year Constructed,Usage Description,Count
0,1850,Assembly,2
1,1850,Business,1
2,1850,Residential,1
3,1852,Assembly,1
4,1853,Assembly,1


In [23]:
usage_dropdown = alt.selection_point(
    fields=["Usage Description"],
    bind=alt.binding_select(
        options=sorted(df_year_usage["Usage Description"].unique()),
        name="Usage: "
    ),
    value="Office"
)

chart2 = (
    alt.Chart(df_year_usage)
    .mark_line(point=True)
    .encode(
        x=alt.X("Year Constructed", title="Year Constructed"),
        y=alt.Y("Count:Q", title="Number of Buildings"),
        color=alt.Color("Usage Description", scale=alt.Scale(scheme="tableau10")),
        size=alt.condition(usage_dropdown, alt.value(4), alt.value(1)),
        tooltip=["Year Constructed", "Usage Description", "Count"]
    )
    .transform_filter(usage_dropdown)
    .add_params(usage_dropdown)
    .properties(width=700, height=400,
                title="Buildings Constructed Over Time by Usage Category")
)

chart2

alt.Chart(...)

In [24]:
chart1.save('../assets/json/hw5_plot1_top10_agency_building_area.json')

In [25]:
chart2.save('../assets/json/hw5_plot2_buildings_constructed_per_year_per_usage.json')